In [ ]:
import ccxt
import os
import datetime

from dotenv import dotenv_values
from pprint import pprint

import ccxt.async_support as ccxt # link against the asynchronous version of ccxt

In [ ]:
config = {
    **dotenv_values(".env.shared"),  # load shared development variables
    **dotenv_values(".env.secret"),  # load sensitive variables
    **os.environ,  # override loaded values with environment variables
}

exchange = ccxt.delta(config)
exchange.verbose = False 

exchange.check_required_credentials()
print(exchange.requiredCredentials) 

In [ ]:
%%capture --no-display

# markets = await exchange.fetchMarkets() # Full reloading
symbol = "BTC/USDT:USDT"
markets = exchange.load_markets()
balance = exchange.fetch_balance()
currencies = exchange.fetch_currencies()
orderbook = exchange.fetch_order_book(symbol)
l2_orderbook = exchange.fetch_l2_order_book(symbol)
fetchTrades = exchange.fetch_trades(symbol)
ticker = exchange.fetch_ticker(symbol)
positions = exchange.fetch_positions()
ledger = exchange.fetch_ledger()

# risk = exchange.fetch_positions_risk(symbol) # not suppoted yet
# transactions = exchange.fetch_transactions() # not supported yet

In [ ]:
# Retrieve only options contract id
_ = {}
for k, v in delta_exchange_markets.items():
    if k.startswith(symbol): _.update({k:v})

btc_options = {}
for k, v in _.items():
    if v["option"]: btc_options.update({k:v})

In [ ]:
raw_btc_options = btc_options["BTC/USDT:USDT-221129:15800:C"]
pprint(raw_btc_options)


In [ ]:
# to get IV/Greeks
a = await exchange.fetchTickers(list(btc_options.keys()))
# to get Index price (Underlying price)
res = await exchange.fetch_ticker(symbol, params={'price':'index'})
index_price = res['last']

a = datetime.datetime.now()
b = datetime.datetime.strptime(raw_btc_options["expiryDatetime"], "%Y-%m-%dT%H:%M:%SZ")

seconds_until_expiration = b-a
seconds_until_expiration

In [ ]:
option_contract = "BTC/USDT:USDT-221129:15800:C"



In [ ]:
"""
Retrieve
IV
index price (Underlying price)
days until expiration
strike price
interest rate
dividend yield (0)
"""

In [ ]:
delta_exchange_tickers

In [ ]:
data_1 = {}
keys_to_select = [
    'id',
    'optionType',
    'strike',
    'base',
    'active',
    'quote',
    'taker',
    'maker',
    'percentage',
    'precision',
    'contractsize',
    'option',
    'tiers',
    'product_specs',
    'spot_index',
    'limits',
]
for k,v in raw_data.items():
    if k in keys_to_select: data_1.update({k:v})
data_1["expiryDatetime"] = seconds_until_expiration

In [ ]:
pprint(exchange.markets['BTC/USDT:USDT-221129:15300:C'])

In [ ]:
data_1

In [ ]:
pprint(data['info'])
select = [
    'annualized_funding',
    'liquidation_penalty_factor',
    'maintenance_margin',
    'maintenance_margin_scaling_factor',
    'maker_commission_rate',
    'max_leverage_notional',
    'notional_type',
    'position_size_limit',
    'product_specs',
    
]

In [ ]:
'optionType': 'call',
 'strike': 15800.0,
'base': 'BTC',
 'quote': 'USDT',
 'contractsize': 0.001,
'option' : True
'product_specs': {'backup_vol_expiry_time': '31536000',
   'max_deviation_from_external_vol': '0.75',
   'max_lower_deviation_from_external_vol': '0.75',
   'max_upper_deviation_from_external_vol': '0.5',
   'max_volatility': '3',
   'min_volatility': '0.2',
   'premium_commission_rate': '0.1',
   'vol_calculation_method': 'orderbook',
   'vol_expiry_time': '31536000'},
'tiers': {'taker': [[0, 0.0015],
   [100, 0.0013],
   [250, 0.0013],
   [1000, 0.001],
   [5000, 0.0009],
   [10000, 0.00075],
   [20000, 0.00065]],
  'maker': [[0, 0.001],
   [100, 0.001],
   [250, 0.0009],
   [1000, 0.00075],
   [5000, 0.0006],
   [10000, 0.0005],
   [20000, 0.0005]]},
   

     'spot_index': {'config': {'impact_size': {'max_impact_size': '150000',
     'min_impact_size': '5000',
     'step_value': '5000'},
    'quoting_asset': 'USDT',
    'service_id': '1',
    'underlying_asset': 'BTC'},
   'constituent_exchanges': [{'exchange': 'binance',
     'health_interval': '60',
     'health_priority': '1',
     'weight': '1'},
    {'exchange': 'huobi',
     'health_interval': '60',
     'health_priority': '2',
     'weight': '1'},
    {'exchange': 'okex',
     'health_interval': '60',
     'health_priority': '3',
     'weight': '1'}],
   'constituent_indices': None,
   'description': 'Bitcoin index from binance, huobi and okex',
   'health_interval': '300',
   'id': '2',
   'impact_size': '1.000000000000000000',
   'index_type': 'spot_pair',
   'is_composite': False,
   'price_method': 'orderbook',
   'quoting_asset_id': '5',
   'symbol': '.DEXBTUSDT',
   'tick_size': '0.500000000000000000',
   'underlying_asset_id': '2'}